## Toxic: TF-IDF/NMF

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.pipeline import Pipeline

from time import time

In [2]:
df = pd.read_pickle('../data/toxictrain.pkl')
print(df.shape)
df.head()

(159571, 19)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,category,rating,comment_text_s,comment_text_f,token_clean,sent_token,polarity_comment,polarity_comment_s,word_count,char_count,char_count_s
idx,,,,,,,,,,,,,,,,,,,
3a4c7758fad18de3,3a4c7758fad18de3,", I hope your retarded kids get anal raped and...",1,1,1,1,1,1,ToSTObThInIH,6,", I hope your retarded kids get anal raped and...",", hope retarded kids get anal raped murdered h...","[i, hope, your, retarded, kids, get, anal, rap...","[, I hope your retarded kids get anal raped an...",0.00,0.00,94,494,494
24d2b50726b67167,24d2b50726b67167,I am going to murder ZimZalaBim ST47 for being...,1,1,1,1,1,1,ToSTObThInIH,6,I am going to murder ZimZalaBim ST47 for being...,am going to murder ZimZalaBim ST47 being evil ...,"[i, am, going, to, murder, zimzalabim, st47, f...",[I am going to murder ZimZalaBim ST47 for bein...,0.00,0.00,12,68,68
c586b7a2fd575b13,c586b7a2fd575b13,"Shut up you asswipe, we don't care. I'll decap...",1,1,1,1,1,1,ToSTObThInIH,6,"Shut up you asswipe, we don't care. I'll decap...","Shut up asswipe , do n't care . 'll decapitate...","[shut, up, you, asswipe, we, dont, care, ill, ...","[Shut up you asswipe, we don't care., I'll dec...",-0.25,-0.25,24,3890,1000
77d84b1321c22d9a,77d84b1321c22d9a,"LGBT \n\nyou little fuck , are you a fag , tha...",1,1,1,1,1,1,ToSTObThInIH,6,"LGBT \n\nyou little fuck , are you a fag , tha...","LGBT little fuck , are fag , piece shit page i...","[lgbt, you, little, fuck, are, you, a, fag, th...","[LGBT \n\nyou little fuck , are you a fag , th...",0.00,0.00,56,280,280
1368c10281978876,1368c10281978876,"You're a stupid cunt \n\nFuck you dumb arse, y...",1,1,1,1,1,1,ToSTObThInIH,6,"You're a stupid cunt \n\nFuck you dumb arse, y...","'re stupid cunt Fuck dumb arse , mum has hairy...","[youre, a, stupid, cunt, fuck, you, dumb, arse...","[You're a stupid cunt \n\nFuck you dumb arse, ...",0.00,0.00,59,278,278


In [3]:
data_samples = df.comment_text

In [4]:
# data_samples

In [5]:
n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20

class_list = ['clean','toxic', 'severe_toxic', 'obscene', 'threat','insult', 'identity_hate']
class_tfidfnmf = {}

def print_top_words(model, feature_names, n_top_words):
    listy = []
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
#         class_tfidfnmf[model] = message
        listy.append(message)
    class_tfidfnmf[model] = listy

In [12]:
for item in class_list:
    listy = []
    if item == 'clean':
        data = df[df['rating']==0]
    else:
        data = df[df[item]==1]

    pipe = Pipeline([('vect',CountVectorizer(ngram_range=(1, 3),  stop_words='english', max_df=0.02, token_pattern="\\b[a-z][a-z]+\\b")),
                ('tfidf',TfidfVectorizer(max_df=0.95, min_df=1,max_features=n_features,stop_words='english'))])
 
    # Use tf-idf features for NMF.
    print("Extracting tf-idf features for NMF...")
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1,
                                       max_features=n_features,
                                       stop_words='english')
    t0 = time()
    tfidf = tfidf_vectorizer.fit_transform(data.comment_text_s)
    print("done in %0.3fs." % (time() - t0))

    # Use tf (raw term count) features for LDA.
    print("Extracting tf features for LDA...")
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=1,
                                    max_features=n_features,
                                    stop_words='english')
    t0 = time()
    tf = tf_vectorizer.fit_transform(data.comment_text_s)
    print("done in %0.3fs." % (time() - t0))
    print()

    # Fit the NMF model
    print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
          "n_samples=%d and n_features=%d..."
          % (n_samples, n_features))
    t0 = time()
    nmf = NMF(n_components=n_components, random_state=1,
              alpha=.1, l1_ratio=.5).fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model (Frobenius norm):")
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
#     listy.append(tfidf_feature_names)
    print_top_words(nmf, tfidf_feature_names, n_top_words)

    # Fit the NMF model
    print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
          "tf-idf features, n_samples=%d and n_features=%d..."
          % (n_samples, n_features))
    t0 = time()
    nmf = NMF(n_components=n_components, random_state=1,
              beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
              l1_ratio=.5).fit(tfidf)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()
#     listy.append(tfidf_feature_names)
    print_top_words(nmf, tfidf_feature_names, n_top_words)

    print("Fitting LDA models with tf features, "
          "n_samples=%d and n_features=%d..."
          % (n_samples, n_features))
    lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)
    t0 = time()
    lda.fit(tf)
    print("done in %0.3fs." % (time() - t0))

    print("\nTopics in LDA model:")
    tf_feature_names = tf_vectorizer.get_feature_names()
#     listy.append(tfidf_feature_names)
    print_top_words(lda, tf_feature_names, n_top_words)
#     class_tfidfnmf[item] = listy
#     ct = pd.DataFrame(class_tfidfnmf)

Extracting tf-idf features for NMF...
done in 16.653s.
Extracting tf features for LDA...
done in 11.332s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 19.585s.

Topics in NMF model (Frobenius norm):
Topic #0: just don know think like people time ve good say really make way does right ll source wp want said
Topic #1: talk redirect contribs archive list comment pages comments wp email discussion ve contributions thank review film reply utc template preceding
Topic #2: blocked editing continue vandalize did edit pages stop ip edits disruptive wikipedia account vandalism address block shared warning make content
Topic #3: deletion speedy deleted tag subject articles criteria notable placed note notability guidelines wikipedia indicate contest speedily adding explaining notice add
Topic #4: article section information sources needs added references articles wp add link removed main source subject written history created does new

done in 0.091s.

Topics in NMF model (Frobenius norm):
Topic #0: fuck shut wikipedia page want fag life mother stop leave nigger fuckin good hey yo bitches people stupid ll bastard
Topic #1: fucking page stop wikipedia moron mother don just life know hope people fucker like stupid gay leave did loser kill
Topic #2: suck cock balls fag lick hey nazi dikkkkk hate dicks prick kiss hairy pussy penis dick sucking filthy sucker mexicans
Topic #3: bitch son little fuckin motherfucker hey stupid ur hell bastard prick gonna make shut billcj ill damn know ya time
Topic #4: asshole fuckin just sandbox motherfucker hell motherfucking mother ur like block life stupid scum stick war wife trade sister complete
Topic #5: cunt stupid die whore mother fat fucker block did bastard dumb hairy head ugly gay sad big stop im trade
Topic #6: shit piece eat die hell little like dumb rot useless burn life wikipedia don jew motherfucker deleting hope cocksucking man
Topic #7: faggot nigger gay hey cocksucker ur 

done in 0.206s.

Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: shit fucking kill just piece think way fuck son hunt hope hello like bitch asshole said time ugly life hit
Topic #1: die hope eat wish hell hate cunt planet dog cancer hi crap soon understand good inshallah dies free whore edie
Topic #2: kill going suggest murder unblock slowly nerd painfully die jews retarder killing said blow disgusting torture grotesque user transsexual homosexual
Topic #3: going watch fucking life shot laugh face head splatter shoot time punch ground person feed little knife homosexual im faggot
Topic #4: ass page ll gonna kick yo ur beat little know fat gay talk face send block ip dont rape faggot
Topic #5: death like destroy deserve right hate things world just really new wait sure war happen look yes bad children good
Topic #6: stupid fuck motherfucker ass suck bitch cut head dick hope faggot fuckin gross stay hours shithead sieg like old pussy
Topic #7: stop rape house fuc

done in 0.424s.

Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: fucking life wikipedia fuck stupid stop right cunt like don think hope ll edit just page idiot block retarded world
Topic #1: gay ur sex likes ugly homosexuality pedophile homophobic ve guys gays warning user john probably men www barnstar maybe rude
Topic #2: nigger stop oh baby sand lol retard thank ll lover house got steal fucked editor 35 loving article 48 whites
Topic #3: faggot hey huge hi nig total ban hello guy internet sean cocksucking website quit knows gay obvious sweet says movie
Topic #4: fuck fucker shut mother muslim nigga whore gay fuckin yo german vandalism ha nig fucked niggaz lost site dead 14
Topic #5: like just people jews don racist niggers want know make good need does wikipedia white man really time world think
Topic #6: ass bitch homosexual like homo page fuckin little penis cock big ur did fucker going ya im sucks hey sucker
Topic #7: shit nigga piece say stop im kill whi

In [13]:
ct = pd.DataFrame(class_tfidfnmf)
ct

,"NMF(alpha=0.1, beta_loss='frobenius', init=None, l1_ratio=0.5, max_iter=200, n_components=10, random_state=1, shuffle=False, solver='cd', tol=0.0001, verbose=0)","NMF(alpha=0.1, beta_loss='kullback-leibler', init=None, l1_ratio=0.5, max_iter=1000, n_components=10, random_state=1, shuffle=False, solver='mu', tol=0.0001, verbose=0)","LatentDirichletAllocation(batch_size=128, doc_topic_prior=None, evaluate_every=-1, learning_decay=0.7, learning_method='online', learning_offset=50.0, max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1, random_state=0, topic_word_prior=None, total_samples=1000000.0, verbose=0)","NMF(alpha=0.1, beta_loss='frobenius', init=None, l1_ratio=0.5, max_iter=200, n_components=10, random_state=1, shuffle=False, solver='cd', tol=0.0001, verbose=0)","NMF(alpha=0.1, beta_loss='kullback-leibler', init=None, l1_ratio=0.5, max_iter=1000, n_components=10, random_state=1, shuffle=False, solver='mu', tol=0.0001, verbose=0)","LatentDirichletAllocation(batch_size=128, doc_topic_prior=None, evaluate_every=-1, learning_decay=0.7, learning_method='online', learning_offset=50.0, max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1, random_state=0, topic_word_prior=None, total_samples=1000000.0, verbose=0)","NMF(alpha=0.1, beta_loss='frobenius', init=None, l1_ratio=0.5, max_iter=200, n_components=10, random_state=1, shuffle=False, solver='cd', tol=0.0001, verbose=0)","NMF(alpha=0.1, beta_loss='kullback-leibler', init=None, l1_ratio=0.5, max_iter=1000, n_components=10, random_state=1, shuffle=False, solver='mu', tol=0.0001, verbose=0)","LatentDirichletAllocation(batch_size=128, doc_topic_prior=None, evaluate_every=-1, learning_decay=0.7, learning_method='online', learning_offset=50.0, max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1, random_state=0, topic_word_prior=None, total_samples=1000000.0, verbose=0)","NMF(alpha=0.1, beta_loss='frobenius', init=None, l1_ratio=0.5, max_iter=200, n_components=10, random_state=1, shuffle=False, solver='cd', tol=0.0001, verbose=0)",...,"LatentDirichletAllocation(batch_size=128, doc_topic_prior=None, evaluate_every=-1, learning_decay=0.7, learning_method='online', learning_offset=50.0, max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1, random_state=0, topic_word_prior=None, total_samples=1000000.0, verbose=0)","NMF(alpha=0.1, beta_loss='frobenius', init=None, l1_ratio=0.5, max_iter=200, n_components=10, random_state=1, shuffle=False, solver='cd', tol=0.0001, verbose=0)","NMF(alpha=0.1, beta_loss='kullback-leibler', init=None, l1_ratio=0.5, max_iter=1000, n_components=10, random_state=1, shuffle=False, solver='mu', tol=0.0001, verbose=0)","LatentDirichletAllocation(batch_size=128, doc_topic_prior=None, evaluate_every=-1, learning_decay=0.7, learning_method='online', learning_offset=50.0, max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1, random_state=0, topic_word_prior=None, total_samples=1000000.0, verbose=0)","NMF(alpha=0.1, beta_loss='frobenius', init=None, l1_ratio=0.5, max_iter=200, n_components=10, random_state=1, shuffle=False, solver='cd', tol=0.0001, verbose=0)","NMF(alpha=0.1, beta_loss='kullback-leibler', init=None, l1_ratio=0.5, max_iter=1000, n_components=10, random_state=1, shuffle=False, solver='mu', tol=0.0001, verbose=0)","LatentDirichletAllocation(batch_size=128, doc_topic_prior=None, evaluate_every=-1, learning_decay=0.7, learning_method='online', learning_offset=50.0, max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001, n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1, random_state=0, topic_word_prior=None, total_samples=1000000.0, verbose=0)","NMF(alpha=0.1, beta_loss='frobenius', init=None, l1_ratio=0.5, max_iter=200, n_components=10, ran